## Cloning the Repo and installing the requirements

In [1]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2615, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 2615 (delta 91), reused 204 (delta 71), pack-reused 2346 (from 1)
Receiving objects: 100% (2615/2615), 69.71 MiB | 10.81 MiB/s, done.
Resolving deltas: 100% (1174/1174), done.


In [2]:
%cd DeOldify

/content/DeOldify


In [3]:
!pip install -r requirements.txt
!pip install -r requirements-colab.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Using cached imgaug-0.2.6.tar.gz (631 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654003 sha256=0616c9b3e04078930aec21b6ebedfd4d1a590faf7e81ebf84a90b9cb8d91b6ad
  Stored in directory: /root/.cache/pip/wheels/cb/c7/a6/2d7a113c4885dc0f4eacd8f41095763181c0b9a18223ac7533
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.4.0
    Uninstalling imgaug-0.4.0:
      Successfully uninstalled imgaug-0.4.0


## Extracting the dataset

In [4]:
import os
import zipfile

zip_path = '/content/archive.zip'
extract_path = 'skin_tone_dataset'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset extracted to '{extract_path}'")


Dataset extracted to 'skin_tone_dataset'


## Multiplying the data and splitting it into two, converting half of it into grayscale

In [8]:
import os
import cv2
import shutil

# Define the path to the original black images
black_images_dir = '/content/DeOldify/skin_tone_dataset/train/Black' # Adjust path if needed
greyscale_dir = 'skin_tone_dataset/train/Greyscale'
colourised_dir = 'skin_tone_dataset/train/Colourised'

# Create directories for Greyscale and Colourised images
os.makedirs(greyscale_dir, exist_ok=True)
os.makedirs(colourised_dir, exist_ok=True)

# Loop through all images in the black dataset
for img_name in os.listdir(black_images_dir):
    if img_name.endswith(('.png', '.jpg', '.jpeg')):  # Filter for image files
        img_path = os.path.join(black_images_dir, img_name)

        # Read the image
        image = cv2.imread(img_path)

        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Save the grayscale image
        gray_save_path = os.path.join(greyscale_dir, img_name)
        cv2.imwrite(gray_save_path, gray_image)

        # Copy the original image to Colourised folder
        colorised_save_path = os.path.join(colourised_dir, img_name)
        shutil.copy(img_path, colorised_save_path)

print("Grayscale images have been created and original images copied.")


Grayscale images have been created and original images copied.


## Splitting Data Into Train and Validation

In [9]:
from sklearn.model_selection import train_test_split

# Define paths for validation datasets
greyscale_val_dir = 'skin_tone_dataset/val/Greyscale'
colourised_val_dir = 'skin_tone_dataset/val/Colourised'

# Create validation directories
os.makedirs(greyscale_val_dir, exist_ok=True)
os.makedirs(colourised_val_dir, exist_ok=True)

# Get all images
greyscale_images = os.listdir(greyscale_dir)
colourised_images = os.listdir(colourised_dir)

# Split the dataset (80% train, 20% validation)
greyscale_train, greyscale_val = train_test_split(greyscale_images, test_size=0.2, random_state=42)
colourised_train, colourised_val = train_test_split(colourised_images, test_size=0.2, random_state=42)

# Move images to train and validation directories
for img_name in greyscale_train:
    shutil.move(os.path.join(greyscale_dir, img_name), os.path.join(greyscale_dir, img_name))
for img_name in colourised_train:
    shutil.move(os.path.join(colourised_dir, img_name), os.path.join(colourised_dir, img_name))

for img_name in greyscale_val:
    shutil.move(os.path.join(greyscale_dir, img_name), greyscale_val_dir)
for img_name in colourised_val:
    shutil.move(os.path.join(colourised_dir, img_name), colourised_val_dir)

print("Dataset has been split into training and validation sets.")


Dataset has been split into training and validation sets.


## Custom Loss Function

In [10]:
import tensorflow as tf

def custom_hue_loss(y_true, y_pred):
    # Calculate the mean squared error
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))

    # Extract hue from the predicted and true images
    y_true_hsv = tf.image.rgb_to_hsv(y_true)
    y_pred_hsv = tf.image.rgb_to_hsv(y_pred)

    # Example: Focus on hue channel (0 index in HSV)
    hue_true = y_true_hsv[:, :, :, 0]  # Hue of true images
    hue_pred = y_pred_hsv[:, :, :, 0]  # Hue of predicted images

    # Calculate hue loss (this can be weighted based on your needs)
    hue_loss = tf.reduce_mean(tf.square(hue_true - hue_pred))

    # Combine losses (you can adjust the weighting)
    total_loss = mse_loss + 0.5 * hue_loss  # Adjust the weight for hue loss
    return total_loss


2024-10-02 09:48:58.653578: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 09:48:58.920814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 09:48:58.999887: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 09:48:59.418288: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 09:49:02.208329: W tensorflow/compiler/tf2

## Downloading the Model

In [22]:
%cd /content/DeOldify

/content/DeOldify


In [23]:
# Step 4: Download the pre-trained model
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

--2024-10-02 09:55:46--  https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth
Resolving data.deepai.org (data.deepai.org)... 185.93.1.251, 2400:52e0:1a00::894:1
Connecting to data.deepai.org (data.deepai.org)|185.93.1.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255144681 (243M) [application/octet-stream]
Saving to: ‘./models/ColorizeArtistic_gen.pth’

./models/ColorizeAr 100%[===================>] 243.32M  5.01MB/s    in 49s     

2024-10-02 09:56:36 (4.92 MB/s) - ‘./models/ColorizeArtistic_gen.pth’ saved [255144681/255144681]



## Fine Tuning the Model

In [27]:
!pip install --upgrade setuptools wheel

  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
Using cached setuptools-75.1.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4


In [26]:
# prompt: Write code to install the following missing modules that can be found in the cloned repository;
# from deoldify.generators import Generator
# from deoldify.device_id import DeviceId
# from deoldify.visualize import *
# from deoldify.utils import *

!pip install -e ./


Obtaining file:///content/DeOldify
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [28]:

import torch
import deoldify
from deoldify import *
import fastai
from fastai import *
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths to the training and validation datasets
train_greyscale_dir = '/content/DeOldify/skin_tone_dataset/train/Greyscale'
train_colourised_dir = '/content/DeOldify/skin_tone_dataset/train/Colourised'
val_greyscale_dir = '/content/DeOldify/skin_tone_dataset/val/Greyscale'
val_colourised_dir = '/content/DeOldify/skin_tone_dataset/val/Colourised'

# Define the image size
image_size = 256

# Define a custom dataset class
class ColorizationDataset(Dataset):
    def __init__(self, greyscale_dir, colourised_dir, transform=None):
        self.greyscale_dir = greyscale_dir
        self.colourised_dir = colourised_dir
        self.greyscale_files = os.listdir(greyscale_dir)
        self.colourised_files = os.listdir(colourised_dir)
        self.transform = transform

    def __len__(self):
        return len(self.greyscale_files)

    def __getitem__(self, idx):
        greyscale_path = os.path.join(self.greyscale_dir, self.greyscale_files[idx])
        colourised_path = os.path.join(self.colourised_dir, self.colourised_files[idx])

        greyscale_image = Image.open(greyscale_path).convert('RGB')
        colourised_image = Image.open(colourised_path).convert('RGB')

        if self.transform:
            greyscale_image = self.transform(greyscale_image)
            colourised_image = self.transform(colourised_image)

        return greyscale_image, colourised_image

# Create data transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

# Create training and validation datasets
train_dataset = ColorizationDataset(train_greyscale_dir, train_colourised_dir, transform=transform)
val_dataset = ColorizationDataset(val_greyscale_dir, val_colourised_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load the pre-trained model
model_path = '/content/DeOldify/models/ColorizeArtistic_gen.pth'
generator = Generator().to(device)
generator.load_state_dict(torch.load(model_path, map_location=device))

# Define the optimizer and loss function
optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)
criterion = custom_hue_loss  # Use your custom loss function

# Training loop
num_epochs = 10  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    generator.train()
    for i, (greyscale_images, colourised_images) in enumerate(train_loader):
        greyscale_images = greyscale_images.to(device)
        colourised_images = colourised_images.to(device)

        optimizer.zero_grad()
        predicted_images = generator(greyscale_images)
        loss = criterion(colourised_images, predicted_images)
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Validation loop (optional)
    generator.eval()
    with torch.no_grad():
        # Perform validation and print metrics
        pass  # Add your validation code here

# Save the fine-tuned model
torch.save(generator.state_dict(), '/content/DeOldify/fine_tuned_model.pth')
print("Fine-tuning complete. Model saved.")


ImportError: cannot import name 'Generator' from 'deoldify.generators' (/content/DeOldify/deoldify/generators.py)

In [6]:
from deoldify.model import ColorizeArtistic  # Adjust this import as needed

def create_model(model_name, pretrained=True):
    if model_name == 'ColorizeArtistic':
        model = ColorizeArtistic()  # Initialize your model architecture here

        # Load pre-trained weights if specified
        if pretrained:
            model.load_state_dict(torch.load('./models/ColorizeArtistic_gen.pth', map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        return model
    else:
        raise ValueError(f"Model '{model_name}' is not recognized. Please check the model name.")

# Step 10: Load the pre-trained model
model = create_model('deoldify.model.ColorizeArtistic', pretrained=False)
model.load_state_dict(torch.load('./models/ColorizeArtistic_gen.pth', map_location=device))
model.to(device)

# Step 11: Create a Learner
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=[accuracy])

# Step 12: Fine-tune the model
learn.load('ColorizeArtistic_gen')  # Load pre-trained weights if necessary
learn.fine_tune(5, base_lr=1e-3)  # Fine-tune the model for 5 epochs

# Step 13: Save the fine-tuned model
learn.save('FineTuned_ColorizeArtistic')

print("Fine-tuning complete!")

ModuleNotFoundError: No module named 'deoldify'

In [17]:
# This assumes you have a proper training script in DeOldify.
# You will need to modify the training script to include your dataset path and loss function.

!python '/content/DeOldify/fastai/train.py' --data_path skin_tone_dataset/train --val_path skin_tone_dataset/val --num_epochs 20 --batch_size 16


Traceback (most recent call last):
  File "/content/DeOldify/fastai/train.py", line 2, in <module>
    from .torch_core import *
ImportError: attempted relative import with no known parent package
